In [2]:
!pip install mujoco mujoco-python-viewer pyvirtualdisplay opencv-python mediapy
!sudo apt-get install xvfb
!pip install xvfbwrapper

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/RL/code') # "RL" 폴더 이름 수정할 것

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Evaluate `SAC` results on `Snapbot`

In [1]:
import mujoco,torch,os
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from snapbot_env import SnapbotMarkovDecisionProcessClass
from sac import ReplayBufferClass,ActorClass,CriticClass,get_target
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Initialize `Snapbot` environment

In [2]:
def np2torch(x_np,device): return torch.tensor(x_np,dtype=torch.float32,device=device)
def torch2np(x_torch): return x_torch.detach().cpu().numpy()
print ("Ready.")

Ready.


In [3]:
xml_path = '../asset/snapbot/scene_snapbot.xml'
env = MuJoCoParserClass(name='Snapbot',rel_xml_path=xml_path,VERBOSE=False)
mdp = SnapbotMarkovDecisionProcessClass(env,HZ=50,history_total_sec=0.2,history_intv_sec=0.1,VERBOSE=True)

[Snapbot] Instantiated
   [info] dt:[0.0200] HZ:[50], env-HZ:[500], mujoco_nstep:[10], state_dim:[35], o_dim:[70], a_dim:[8]
   [history] total_sec:[0.20]sec, n:[10], intv_sec:[0.10]sec, intv_tick:[5]
   [history] ticks:[0 5]


### Check pth files

In [4]:
dir_path = '../result/weights/sac_%s/'%(mdp.name.lower())
filenames = sorted(os.listdir(path=dir_path))
print ("We have [%d] files:"%(len(filenames)))
print (filenames)

We have [21] files:
['episode_0.pth', 'episode_100.pth', 'episode_150.pth', 'episode_200.pth', 'episode_250.pth', 'episode_300.pth', 'episode_350.pth', 'episode_400.pth', 'episode_450.pth', 'episode_50.pth', 'episode_500.pth', 'episode_550.pth', 'episode_600.pth', 'episode_650.pth', 'episode_700.pth', 'episode_750.pth', 'episode_800.pth', 'episode_850.pth', 'episode_900.pth', 'episode_950.pth', 'episode_999.pth']


### Load

In [5]:
epi_idx  = 999
pth_path = '../result/weights/sac_%s/episode_%d.pth'%(mdp.name.lower(),epi_idx)
if os.path.exists(pth_path): print ("We will use [%s]."%(pth_path))
else: print ("ERROR:[%s] DOES NOT EXIST!"%(pth_path))

We will use [../result/weights/sac_snapbot/episode_999.pth].


### Loop

In [6]:
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(400, 300))
# display.start()

video = []

# Simulator
max_epi_sec  = 10.0 # maximum episode length in second
max_epi_tick = int(max_epi_sec*mdp.HZ) # maximum episode length in tick
print("max_epi_tick: ", max_epi_tick)
# Actor
device       = 'cpu' # cpu / mps / cuda
max_torque   = 2.0
actor = ActorClass(
    obs_dim=mdp.o_dim,h_dims=[256,256],out_dim=mdp.a_dim,max_out=max_torque,device=device).to(device)
# Load pth
actor.load_state_dict(torch.load(pth_path,map_location=device))
# Run
mdp.init_viewer() # init viewer
s = mdp.reset() # reset state
reward_total = 0.0
for tick in range(max_epi_tick):
    a,_ = actor(np2torch(s,device=device),SAMPLE_ACTION=False) # get action
    s_prime,reward,done,info = mdp.step(torch2np(a),max_time=max_epi_sec) # step
    if (tick % 2) == 0:
        mdp.render(TRACK_TORSO=True,PLOT_WORLD_COORD=True,PLOT_TORSO_COORD=True,
                   PLOT_SENSOR=True,PLOT_CONTACT=True,PLOT_TIME=True)
    reward_total += reward
    s = s_prime
    if not mdp.is_viewer_alive(): break

    if (mdp.tick % 5) == 0:
      print("current tick: ", mdp.tick)
      scene_img = mdp.grab_image(resize_rate=0.25)
      video.append(scene_img)


mdp.close_viewer() # close viewer
x_diff = mdp.env.get_p_body('torso')[0]
print ("[Eval] time:[%.1f]sec reward:[%.3f] x_diff:[%.3f]m epi_len:[%d/%d]"%
       (mdp.get_sim_time(),reward_total,x_diff,tick,max_epi_tick))

max_epi_tick:  500
current tick:  5
current tick:  10
current tick:  15
current tick:  20
current tick:  25
current tick:  30
current tick:  35
current tick:  40
current tick:  45
current tick:  50
current tick:  55
current tick:  60
current tick:  65
current tick:  70
current tick:  75
current tick:  80
current tick:  85
current tick:  90
current tick:  95
current tick:  100
current tick:  105
current tick:  110
current tick:  115
current tick:  120
current tick:  125
current tick:  130
current tick:  135
current tick:  140
current tick:  145
current tick:  150
current tick:  155
current tick:  160
current tick:  165
current tick:  170
current tick:  175
current tick:  180
current tick:  185
current tick:  190
current tick:  195
current tick:  200
current tick:  205
current tick:  210
current tick:  215
current tick:  220
current tick:  225
current tick:  230
current tick:  235
current tick:  240
current tick:  245
current tick:  250
current tick:  255
current tick:  260
current tick:

In [7]:
import cv2, os

shape = video[0].shape
video_folder = ("../video")
if not os.path.isdir(video_folder): os.mkdir(video_folder)
video_out = cv2.VideoWriter(os.path.join(video_folder, "snapbot_sac.mp4"), cv2.VideoWriter_fourcc('F','M','P','4'), 20, (shape[1], shape[0]), True)
for i in range(len(video)):
  video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
  video_out.write(video_frame)
video_out.release()

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
import mediapy
video_path = os.path.join(video_folder, "snapbot_sac.mp4")
mediapy.show_video(mediapy.read_video(video_path))
